In [1]:
import sys
import os
current_dir = os.getcwd()  
project_root = os.path.dirname(current_dir)  
sys.path.insert(0, project_root) 
import numpy as np
import pandas as pd
from pathlib import Path
from pycaret.regression import load_model, predict_model
from src.config.config import Config
from src.data.data_loader import DataLoader
from src.visualization.visualizer import Visualizer

def predict_extdata(test_data: pd.DataFrame, model_path: str):
    """预测测试集结果并返回评估指标

    Args:
        test_data: 测试数据集
        model_path: 模型文件路径
        test_type: 测试类型，'country' 或 'time'

    Returns:
        dict: 包含预测结果和评估指标的字典
    """
    target_column = Config.DATA_CONFIG['target_column']
    method = Config.FEATURE_CONFIG['target_transform_method']
    transformed_column = f'{target_column}_{method}'
    target_column_pred = f'{target_column}_pred'

    # 移除目标列和转换后的目标列
    test_data = test_data.drop([target_column, transformed_column], axis=1, errors='ignore')

    # 加载模型
    model = load_model(model_path)
    
    # 预测
    predictions = predict_model(model, data=test_data)

    # 处理预测结果
    test_results = predictions.copy()
    test_results[target_column_pred] = np.expm1(predictions['prediction_label'])

    return test_results

In [2]:
df = pd.read_csv(Path(Config.PATH_CONFIG['features_dir'])  / 'prediction_data.csv')

# 模型路径
model_path = os.path.join(Config.PATH_CONFIG['models_dir'], 'gbr_250412_192557')

# Predict application data set
print('\n Application data set predictions:')
results = predict_extdata(df, model_path)


 Application data set predictions:
Transformation Pipeline and Model Successfully Loaded


In [3]:
# 确保预测结果目录存在
prediction_dir = Path(Config.PATH_CONFIG['prediction_dir'])
prediction_dir.mkdir(parents=True, exist_ok=True)

# 生成文件名（使用当前时间戳）
from datetime import datetime
timestamp = datetime.now().strftime('%d%m%y_%H%M%S')
output_file = prediction_dir / f'iw_predictions_{timestamp}.csv'

# 保存预测结果
results.to_csv(output_file, index=False)
print(f'预测结果已保存至: {output_file}')

预测结果已保存至: e:\code\jupyter\固废产生\SW-Prediction\iw\src\results\prediction\iw_predictions_120425_194829.csv
